In [ ]:
from random import choice

import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt

from sklearn import linear_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [3]:
# If you do nothave scikit-learn installed, uncomment the following line
# !conda install -y -c conda-forge scikit-learn

If you have issues with VSCode notebook cell outputs being truncated:

- Go to Settings (via menubar or cmd-, on Mac)
- Search for cell output settings: try @tag:notebookOutputLayout
- Adjust settings, e.g. scrolling, number of lines to display

# Workshop - ML predictions of aqueous solubility

## Introduction

In this workshop, you will get some hands on practice of applying some of the major machine learning (ML) models to a chemical dataset.

### The data

AqSolDB ([Sorkun et al.](https://doi.org/10.1038/s41597-019-0151-1)) is a curated dataset of experimentally-determined aqueous solubility values, with calculated descriptors for the molecules.

The paper gives details on how the data was acquired and processed, and its availability on a number of platforms including [github](https://github.com/mcsorkun/AqSolDB)


### The task

Prepare the data for training and evaulating a set of machine learning models to predict the solubilty of the compounds based on the features supplied (and others if you would like to calculate additional descriptors as features).

You will use scikit-learn to train and evaluate the following models:

**Supervised learning**

- Linear regression
- Logistic regression
- k-Nearest neighbors



**Unsupervised learning** 
:::{note}
There is a separate notebook for this, if we get on to it. It will not be assessed.
:::


- k-Means clustering
- PCA for dimensionality reduction

#### Steps

1. Load the data
2. Perform some EDA to gain initial understanding of the distribution of features and relationships between features, and with the target.

For each model (may require additional stages depending on the model)

3. Prepare the data 
4. Train the model
5. Make predictions
6. Evaluate performance

7. Analyse the performance of the models. Draw conclusions about the chemical problem, e.g. from the feature importances.

## Load the data and perform exploratory analysis

You can perform some initial exploratory analysis of the dataset using some of the methods you saw last week.

In addition to looking for distribution and patterns in the data, look at what the columns actually contain. Some will include metadata about the source of the observation and its processing, which will not be relevant to the target variable.

In [ ]:
# TODO: 
# - Check the data and load into a DataFrame
# - Check the data types
# - Check for missing values
# - Check summary statistics
# - Identify redundant columns

In [ ]:
# TODO:
# - Visualise the data to look for distributions of features, check for outliers
# - Visualise the data to look for correlations
# - Visualise the data to look for relationships between features and target

### Questions:

- Explain your approach to EDA for the dataset. What questions can this process answer and suggest how it can aid the subsequent analysis and modelling.
- What are the most significant correlations in this dataset? Discuss any strong relationships between the features and the target variable that are apparent.
- If you were selecting features from the data, are there any that you would remove? Explain why/why not.


#### Things to consider

- Note the distributions of values of features (e.g. the measures of centre, the magnitude and shape of the distribution and the range of the values).


## 1. Linear regression

The first model we will apply is a [linear regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).

Linear regression models the relationship between input features and a continuous target variable using a linear function.

The function looks like:  

$$
y = w_0 + w_1x_1 + w_2x_2 + \dots + w_nx_n + \epsilon
$$

Where:  
- $ y $ = **Predicted output** (target variable)  
- $ x_1, x_2, \dots, x_n $ = **Input features** (independent variables)  
- $ w_0 $ = **Intercept** (bias term)  
- $ w_1, w_2, \dots, w_n $ = **Coefficients** (weights)  
- $ \epsilon $ = **Error term** (accounts for noise in data)  

The goal is to find weights $w_{i}$ that minimize the error, typically using Ordinary Least Squares (OLS).

It finds a best-fit line by minimising the difference between predictions and actual values, typically using least squares. 

It is widely used for trend analysis, forecasting, and understanding feature impact on outcomes.

### Prepare data

To prepare the data, create a new dataframe containing only the numerical features of the AqSolDB dataset.

#### Separate features and target

You can now separate your data into the features (the predictor variables) and target (the variable you want to predict).

In [ ]:
# TODO:
# - Read the target column into a separate variable
# - Read the feature columns into a different variable - remember to drop the target column



#### Create the training and test sets

Run [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to create separate training and test sets, with 20% of the samples in the test set.

In [ ]:
# TODO Split the data into training (80%) and testing (20%) sets
# and check the size of the resulting datasets



((7985, 17), (1997, 17), (7985,), (1997,))

### Training the model

It is time to train the first ML model.

You will need to create a new LinearRegression model and train it using its `fit` method on the training data's features.

In [ ]:
# TODO: Create a linear regression model


# TODO: Fit the model to the training data


LinearRegression()

### Test the model's performance on unseen data

You can now get the model to predict the solubilities for the subset of data you withheld for the test set.

In [ ]:
# TODO: Predict the solubility of the test set


### Evaluating the model's performance

We can visualise how closely the predicted solubility values for both the training and/or test set match the real values.

:::{hint}
You will need to also generate predictions for the test set if you want to visualise
:::

There are a variety of metrics that can be used to quantify the model's performance. 

One commonly used metric for regression tasks is $r^2$ which expresses how well the model fits the data. It ranges from 0 to 1, with 1 indicating a perfect fit.

In [ ]:
# TODO: Calculate r^2 value is a measure of how well the model fits the data. It ranges from 0 to 1, 
# with 1 indicating a perfect fit.



0.49474578991346907

### Questions

- What other metrics might be useful for evaluating the model's performance? Choose one other metric and calculate it for the model's perform on the test data. Briefly explain the form and meaning of the metric.
- Comment on the performance of the model on the training vs. the test data. Is there anything you can infer from the comparison?
- What information can you gain from the model coefficients? (If you want to do this, you will need to scale the features to compare them - see the [notebook](../book/3-ml_intro/ML_demo) ) How could you use this to improve model or the training process?


## 2. Logistic regression

Logistic regression is used for binary classification: predict to which of two classes an input belongs.

In the context of AqSolDB, we can convert solubility values (logS) into two classes:

Soluble (1): logS above a certain threshold (e.g., logS > -2)
Insoluble (0): logS below the threshold

This allows us to predict solubility as a classification problem.

### Prepare data

Get a copy of the dataframe after you had dropped the non-numeric features.

You will need to add a new target variable based on the current `solubility` column, where the new column value is:

`1` if `logS >= -2`  

`0` if `logS < -2`


In [34]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# TODO: 
# - Create a copy of the original DataFrame with numeric columns only
# - Add a new column with binary solubility values
# - Drop the original solubility column



MolWt   MolLogP     MolMR  HeavyAtomCount  NumHAcceptors  NumHDonors  \
0   392.510   3.95810  102.4454            23.0            0.0         0.0   
1   169.183   2.40550   51.9012            13.0            1.0         1.0   
2   140.569   2.15250   36.8395             9.0            1.0         0.0   
3   756.226   8.11610  200.7106            53.0            6.0         2.0   
4   422.525   2.48540  119.0760            31.0            6.0         0.0   
5   118.179   2.63802   41.2700             9.0            0.0         0.0   
6   170.252   2.67750   47.9918            12.0            1.0         1.0   
7   376.449   0.52840   96.4382            27.0            6.0         4.0   
8   218.202   3.19580   56.2325            16.0            1.0         0.0   
9   342.391   3.49720   93.5020            25.0            5.0         0.0   
10  149.193   1.56510   46.1565            11.0            2.0         0.0   
11  192.258   2.57080   55.9910            14.0            2.0         0.0   
12  733.002  12.05424  225.4848            54.0            6.0         6.0   
13  206.285   2.76170   60.3770            15.0            2.0         0.0   
14  260.255   3.70480   61.5015            18.0            2.0         0.0   
15  154.253   2.67140   49.5078            11.0            1.0         1.0   
16  214.648   2.50032   53.9148            14.0            2.0         1.0   
17  660.282   9.32430  163.3960            35.0            8.0         0.0   
18  180.556   3.35880   36.4540            11.0            0.0         0.0   
19  228.332   2.78090   63.1690            16.0            3.0         0.0   

    NumHeteroatoms  NumRotatableBonds  NumValenceElectrons  NumAromaticRings  \
0              2.0               17.0                142.0               0.0   
1              2.0                0.0                 62.0               2.0   
2              2.0                1.0                 46.0               1.0   
3              7.0               10.0                264.0               6.0   
4              6.0               12.0                164.0               2.0   
5              0.0                1.0                 46.0               1.0   
6              2.0                4.0                 70.0               0.0   
7              6.0                2.0                148.0               0.0   
8              3.0                2.0                 80.0               2.0   
9              5.0               10.0                132.0               2.0   
10             2.0                2.0                 58.0               1.0   
11             2.0                3.0                 76.0               1.0   
12             6.0                0.0                288.0               6.0   
13             2.0                3.0                 82.0               1.0   
14             5.0                6.0                100.0               1.0   
15             1.0                4.0                 64.0               0.0   
16             4.0                3.0                 76.0               1.0   
17            11.0               20.0                208.0               0.0   
18             4.0                0.0                 60.0               1.0   
19             3.0                7.0                 94.0               0.0   

    NumSaturatedRings  NumAliphaticRings  RingCount    TPSA   LabuteASA  \
0                 0.0                0.0        0.0    0.00  158.520601   
1                 0.0                1.0        3.0   29.10   75.183563   
2                 0.0                0.0        1.0   17.07   58.261134   
3                 0.0                0.0        6.0  120.72  323.755434   
4                 4.0                4.0        6.0   56.60  183.183268   
5                 0.0                0.0        1.0    0.00   55.836626   
6                 1.0                1.0        1.0   37.30   73.973655   
7                 3.0                4.0        4.0  115.06  158.135542   
8          

### Separate features and target and test-train split

Follow the same process as for the linear regression and separate the target and feature columns.

Then split the data into training and testing sets. Make sure you run this with `stratify=<name of your target array>`. (What does [`stratify`](https://machinelearningmastery.com/train-test-split-for-evaluating-machine-learning-algorithms#:~:text=stratified%20train-test%20split) do?)

In [ ]:
# TODO:
# - Separate features and target column
# - Split the data into training and testing sets - split first
# - Scale the features using StandardScaler - scale the test and training sets separately


In [45]:
# TODO: 
# - Create a logistic regression model
# - Fit the model to the training data
# - Predict the solubility of the test set
# - Calculate the accuracy of the model



The [`classification report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) provides a set of metrics for classification tasks.

### Questions

- Briefly explain the meaning of the metrics in the classification report.
- Comment on the performance of the regression and classification models. Why might this approach be useful for some types of problems?

## 3. k-NN classification

Over to you for this one. Here is the documentation for sklearn's [`KNeighboursClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

The process follows a very similar process to the models you have already seen.

You can either stick with the binary classification or use the classes described in the AqSolDB paper:


| Category | logS range |
|------------|----------|
|**Highly soluble** | logS > 0 |
|**Soluble** | 0 > logS > -2 |
|**Slightly soluble**  | -2 > logS > -4 |
|**Insoluble** | logS < -4 |


There are a few important points:


1. Make sure you use `stratify` when you split the data and pass it the full target array.
2. You must scale the features using StandardScaler after splitting.
3. k-NN has a hyperparameter, so you will need to use cross-validation to adjust the value of k. There is a quick tutorial [here](https://www.datacamp.com/tutorial/k-nearest-neighbor-classification-scikit-learn)

#### Devise your evaluation strategy for the k-NN model

Scikit-learn has a variety of methods to [measure and present](https://scikit-learn.org/stable/api/sklearn.metrics.html) model performance, e.g.

- classification report 
- confusion matrix

### Summary

As you have worked through this notebook, you have 

- Used exploratory analysis to identify and understand the structure of and trends within a moderately-sized chemical dataset.

- Prepared a dataset to apply predictive modelling.

- Trained, tested and evaluated some frequently-used ML models to predict a chemical property.

In addition to the practical and technical skills you will have acquired in applying machine learning for this task, as part of the process, you have seen that it is important to critically consider how best to use the data you have available to address the scientific problem that you have.

The process of structuring your data, selecting a model, selecting features, etc. can be a highly iterative process. It is important to think critically about how your data is being processed, how the model is learning from it, and how well the model’s predictions align with the real-world problem you are addressing.

Machine learning is not a black-box tool but a structured approach that requires careful decision-making at every stage. This includes selecting appropriate features, choosing a suitable model, and ensuring rigorous evaluation of performance. Model results should not be taken at face value: It is essential to assess accuracy, biases, and generalisation.

By approaching ML critically and iteratively, you can refine your models, improve predictions, and ensure that the insights gained are scientifically meaningful and reliable.

### Final questions

- Based on the models you have trained and tested, how would you decide which model is most appropriate for predicting solubility categories? Consider the evaluation metrics, feature selection, and any limitations you observed.

- Suggest one way that you could you iteratively refine your approach - e.g. adjusting models, features, or preprocessing steps - to improve predictive performance?